<a href="https://colab.research.google.com/github/mzohaibnasir/cLabs_Miscellaneous/blob/main/summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>